<a href="https://colab.research.google.com/github/k-i-wilcox/MATH120_Final_Project_KIW/blob/main/GrainSize_Animation_Working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook takes seived sediment data, calculates the cumulative grain size, and outputs the D10, D16, D50, D84, and D90 for each sample.  Input is a CSV files with the weight of sediment for each sieve size. 

# Import Packages

In [206]:
#!pip install -upgrade gsspread
#!pip install RISE
#!pip install -c plotly plotly-orca
#!pip install -c plotly plotly=5.10.0
#!pip install plotly
#!pip uninstall tornado ## plotly had stoped working give an error refering to tornado. Had to uninstall and reinstall for it to work
#!pip install tornado
#!pip install geopandas
#import time
#import(geopandas)


In [207]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from matplotlib import pyplot as plt
import tornado.ioloop
import sympy as sy
import plotly.express as px # was throwing error with tornado, so I uninstalled and then reinstalled tornado
import time
from google.colab import files
import plotly.graph_objects as go
import csv
import os
import seaborn as sns
import matplotlib.animation as animation
import plotly.express as px
import plotly.graph_objects as go
#import geopandas as gpd
import datetime, math
import io



# BLH-84 Bedload Sample Data

## Dataframe Format

In [208]:
# mount google drive on notebook
from google.colab import drive
drive.mount('/drive')

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)
from gspread.models import Worksheet
from gspread_dataframe import set_with_dataframe

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


To open the desired bedload sample, uncomment the bedload grain size sheet for the meadow you want, then uncomment the specific tab for sample you want to analyze. 

In [209]:
# FORMATTING DATAFRAME
# Open Bedload Grain Size Sheet
# gs0 = gc.open_by_key('1WnHHtdJR-k6nKZFD3F-oagLjsGE2lecYm0F5eSkrsew') # Ahart
gs0 = gc.open_by_key('1WnHHtdJR-k6nKZFD3F-oagLjsGE2lecYm0F5eSkrsew') # Grouse

# Open specific tab
ws = gs0.worksheet('AHU2022/03/31')

# get_all_values gives a list of rows.
rows = ws.get_all_values()
df = pd.DataFrame(rows)

# getting rid of column index to so it can be used in Pandas
df.columns = df.iloc[0]
df = df.iloc[1:]

df = df[["sample_id","mm", 'phi', 'weight']] # df is the name of dataframe you will be working with

# drop notes column
# df2 = df.drop(columns=['notes'])

# export both dataframes
# path = '/drive/MyDrive/Classes/Math_120/Midterm_2/'
# df.to_csv(path + 'unformatted_dataframe.csv')
# df2.to_csv(path + 'formatted_dataframe.csv')
df

,sample_id,mm,phi,weight
1,AHU2022/03/31,8.000,-3.000,0.000
2,AHU2022/03/31,5.657,-2.500,3.356
3,AHU2022/03/31,4.760,-2.251,0.363
4,AHU2022/03/31,2.828,-1.500,1.815
5,AHU2022/03/31,2.000,-1.000,1.813
6,AHU2022/03/31,1.414,-0.500,2.813
7,AHU2022/03/31,1.000,0.000,2.020
8,AHU2022/03/31,0.707,0.500,3.138
9,AHU2022/03/31,0.500,1.000,5.217
10,AHU2022/03/31,0.354,1.500,2.002


In [210]:
df.info() # shows the size of the dataframe

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 1 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sample_id  13 non-null     object
 1   mm         13 non-null     object
 2   phi        13 non-null     object
 3   weight     13 non-null     object
dtypes: object(4)
memory usage: 548.0+ bytes


In [211]:
#len(df.index)
#df.shape[0] <- rows
#df.shape[1] < columns

In [212]:
# convert columns from strings to numeric
# this code has to be run after the dataframe is formatted 
df["weight"] = pd.to_numeric(df["weight"])
df["mm"] = pd.to_numeric(df["mm"])
df["phi"] = pd.to_numeric(df["phi"])

## Calculating Cumulative Fraction Finer

In [213]:
# CALCULATE FRACTION FINER AND CUMALATIVE GRAIN SIZE
# Create two empty columns and append to dataframe
df['fraction_finer'] = np.zeros((13,1)) 
df ['cumulative_finer'] = np.zeros((13,1))
def cumulative_grain (df):
  """Input a dataframe with column headings in the following order: sample_id, 
      sieve size in mm, sieve size in phi, weight of sediment(g) in each sieve.
        Function will find the total weight and calculate and append the 
        fraction finer and cumulative percent finer to the dataframe """
  weight_sum = df['weight'].sum() # finds sum of weight column
  for i in range(13):
    df.iloc[i, 4] = (df.iloc[i,3] / weight_sum) * 100
 
  for i in range(13):
    if i == 0:                     # first row of cumulative finer should always be 100
       df.iloc[i,5] = 100
    
    else:
      df.iloc[i,5] = (df.iloc[i-1,5] - df.iloc[i,4])

cumulative_grain(df)
print(df)

0       sample_id     mm    phi  weight  fraction_finer  cumulative_finer
1   AHU2022/03/31  8.000 -3.000   0.000        0.000000      1.000000e+02
2   AHU2022/03/31  5.657 -2.500   3.356       13.229265      8.677073e+01
3   AHU2022/03/31  4.760 -2.251   0.363        1.430937      8.533980e+01
4   AHU2022/03/31  2.828 -1.500   1.815        7.154683      7.818512e+01
5   AHU2022/03/31  2.000 -1.000   1.813        7.146799      7.103832e+01
6   AHU2022/03/31  1.414 -0.500   2.813       11.088773      5.994954e+01
7   AHU2022/03/31  1.000  0.000   2.020        7.962788      5.198675e+01
8   AHU2022/03/31  0.707  0.500   3.138       12.369915      3.961684e+01
9   AHU2022/03/31  0.500  1.000   5.217       20.565279      1.905156e+01
10  AHU2022/03/31  0.354  1.500   2.002        7.891832      1.115973e+01
11  AHU2022/03/31  0.250  2.000   2.420        9.539577      1.620151e+00
12  AHU2022/03/31  0.212  2.238   0.252        0.993377      6.267739e-01
13  AHU2022/03/31  0.125  3.000   0.15

In [214]:
df # dataframe updated with the fraction finer and cumulative grain size

,sample_id,mm,phi,weight,fraction_finer,cumulative_finer
1,AHU2022/03/31,8.000,-3.000,0.000,0.000000,1.000000e+02
2,AHU2022/03/31,5.657,-2.500,3.356,13.229265,8.677073e+01
3,AHU2022/03/31,4.760,-2.251,0.363,1.430937,8.533980e+01
4,AHU2022/03/31,2.828,-1.500,1.815,7.154683,7.818512e+01
5,AHU2022/03/31,2.000,-1.000,1.813,7.146799,7.103832e+01
6,AHU2022/03/31,1.414,-0.500,2.813,11.088773,5.994954e+01
7,AHU2022/03/31,1.000,0.000,2.020,7.962788,5.198675e+01
8,AHU2022/03/31,0.707,0.500,3.138,12.369915,3.961684e+01
9,AHU2022/03/31,0.500,1.000,5.217,20.565279,1.905156e+01
10,AHU2022/03/31,0.354,1.500,2.002,7.891832,1.115973e+01


In [215]:
# export newly calculate colums to same google sheet
df_to_sheets = df[['fraction_finer','cumulative_finer']]
set_with_dataframe(ws,df_to_sheets,row=1,col=5,include_column_header=True)
#df

## Cumulative Grain Size Curve

In [216]:
# PLOT OF CUMULATIVE GRAIN SIZE AND GRAIN SIZE IN MM
df["mm"] = pd.to_numeric(df["mm"])
fig = px.line(df, x = 'mm', y = 'cumulative_finer', 
               title = "Cumulative Grain Size", labels=dict(cumulative_finer ="Cumulative % Finer", mm =  "Grain Size (mm)"))
fig.update_yaxes(tickvals=[0,10,20,30,40,50,60,70,80,90,100])
#path = '/drive/MyDrive/Classes/Math_120/Midterm_2/'   # have to add path for google colab
#plt.savefig(path + "LGL cumulative grain size curve.png")

In [217]:
fig.show()

In [218]:
# SAVE FIGURE - uncomment to save
path = '/drive/MyDrive/Classes/Math_120/Midterm_2/'
# plt.savefig(path + "LGL cumulative grain size curve.png")

## Grain Size Function

In [251]:
L = len(df) # assigns length of dataframe to L
def grain_size (y1):
  """input the desired percent cumalative finer 
  will output the corresponding grain size in mm"""
  for i in range (L-1): # i has to be one size smaller than the length. 
    if df.iloc[i+1,5] <= y1 <= df.iloc[i,5]: 
      m = (df.iloc[i,5]- df.iloc[i+1,5]) / (df.iloc[i,1] - df.iloc[i+1,1])
      b = df.iloc[i+1,5] - m * df.iloc[i+1,1]
      x = (y1-b)/m
      print("D:", y1,'is equal to', x, "mm")

      if y1 < 0:
        raise Exception('y cannot be less than 0')

      if y1 > 100:
        raise Exception('y cannot be greater than 100')         
      
  return y1, x

In [220]:
grain_size(50) # gives grain size in mm for 

D: 50 is equal to 0.952940726577438 mm


(50, 0.952940726577438)

In [221]:
y_val = [10,16,50,84,90]               # diameter percentiles - will find the corresponding mm

BL_diameter = pd.DataFrame(np.zeros((5,3)))
for i in range(len(y_val)):
  y, x = grain_size(y_val[i])
  BL_diameter.iloc[i, 0] = df.iloc[i, 0] # adds the sample name to df
  BL_diameter.iloc[i, 1] = y              # adds the diameter percentile
  BL_diameter.iloc[i, 2] = x              # adds the diameter in mm
BL_diameter

D: 10 is equal to 0.34135669421487624 mm
D: 16 is equal to 0.4435456943056946 mm
D: 50 is equal to 0.952940726577438 mm
D: 84 is equal to 4.398210380165287 mm
D: 90 is equal to 6.228926579261024 mm


,0,1,2
0,AHU2022/03/31,10.0,0.341357
1,AHU2022/03/31,16.0,0.443546
2,AHU2022/03/31,50.0,0.952941
3,AHU2022/03/31,84.0,4.398210
4,AHU2022/03/31,90.0,6.228927


In [222]:
BL_diameter.columns =['sample', 'diameter', 'D_mm'] # adds names to df columns
BL_diameter

,sample,diameter,D_mm
0,AHU2022/03/31,10.0,0.341357
1,AHU2022/03/31,16.0,0.443546
2,AHU2022/03/31,50.0,0.952941
3,AHU2022/03/31,84.0,4.398210
4,AHU2022/03/31,90.0,6.228927


In [223]:
# Outputs data to the same google worksheet
BL_diameter_to_sheets = BL_diameter[['diameter', 'D_mm']]
set_with_dataframe(ws,BL_diameter_to_sheets,row=1,col=7,include_column_header=True)
#df3

# Pebble Count & Bulk Sample Data

## Bulk Sample

### Dataframe Formatting







Uncomment the bed surface grain size sheet for the meadow you would like analyze. To access the specific sample, uncomment the name under the meadow tab heading. 

In [224]:
# FORMATTING DATAFRAME
# Open Bed Surface Grain Size Sheet
bs0 = gc.open_by_key('1nnCc9azCJu7GB3-T-P6k1NPGTPHQS43JJ9RHv9atFgQ') # Ahart Bulk Sample
# bs0 = gc.open_by_key('1MCwy66A2Ez-aNn2nq2CEucFU_d_c8PhHrylfxlegY4U') # Grouse Bulk Sample
# bs0 = gc.open_by_key('1OepejnoaO2dM4Z9UTl4Zooy6dyMdv2XxhO8rqBrpJlg') # McCreary Bulk Sample

# To open Ahart tab, delete comment 
# ws = bs0.worksheet('AHA22-CP10R-BS1 Total')
# ws = bs0.worksheet('AHA22-CP30R-BS2')
# ws = bs0.worksheet('AHA22-CP45R-BS3')
# ws = bs0.worksheet('AHA22-CP90R-BS4 Total')
ws = bs0.worksheet('AHA22-CP103GR-BS5 Total')

# Open Lower Grouse Tab
# ws = bs0.worksheet('LGR22_CP10R_BS1')
# ws = bs0.worksheet('LGR22_CP30R_BS2')
# ws = bs0.worksheet('LGR22_CP50R_BS3')
# ws = bs0.worksheet('LGR22_CP70R_BS4_TOTAL')
# ws = bs0.worksheet('LGR22_CP90R_BS5')

# Open McCreary Tab
# ws = bs0.worksheet('MCC22-CP70R-BS1_TOTAL')
# ws = bs0.worksheet('MCC22-CP30R-BS2_TOTAL')
# ws = bs0.worksheet('MC22-CP10R_BS3_TOTAL')


# get_all_values gives a list of rows.
rows = ws.get_all_values()
df2 = pd.DataFrame(rows)

# getting rid of column index to so it can be used in Pandas
df2.columns = df2.iloc[0] # update name to new df name
df2 = df2.iloc[1:]  

# only pulling the names of the columns that I want
df2 = df2[["sample_id","mm", 'phi', 'weight']]

# drop notes column, uncomment if needed
# df2 = df.drop(columns=['notes'])

# export both dataframes
# path = '/drive/MyDrive/Classes/Math_120/Midterm_2/'
# df.to_csv(path + 'unformatted_dataframe.csv')
# df2.to_csv(path + 'formatted_dataframe.csv')
# df2

# convert columns from strings to numeric
# this code has to be run after the dataframe is formatted 
df2["weight"] = pd.to_numeric(df2["weight"])
df2["mm"] = pd.to_numeric(df2["mm"])
df2["phi"] = pd.to_numeric(df2["phi"])

df2

,sample_id,mm,phi,weight
1,AHA22-CP103GR-BS5,45.3000,-5.501,0.0
2,AHA22-CP103GR-BS5,31.5000,-4.977,298.9
3,AHA22-CP103GR-BS5,22.4000,-4.485,0.0
4,AHA22-CP103GR-BS5,16.0000,-4.000,0.0
5,AHA22-CP103GR-BS5,11.2000,-3.485,11.0
6,AHA22-CP103GR-BS5,8.0000,-3.000,26.2
7,AHA22-CP103GR-BS5,5.6000,-2.485,138.4
8,AHA22-CP103GR-BS5,4.0000,-2.000,431.1
9,AHA22-CP103GR-BS5,2.8000,-1.485,743.1
10,AHA22-CP103GR-BS5,2.0000,-1.000,1036.7


### Calculating Cumulative Fraction Finer

In [225]:
# CALCULATE FRACTION FINER AND CUMALATIVE GRAIN SIZE
# find the df length (# of rows) and assign to l
# insert into loop
L = len(df2.index)
# Create two empty columns and append to dataframe
df2 ['fraction_finer'] = np.zeros((L,1)) # L = the length of df2
df2 ['cumulative_finer'] = np.zeros((L,1))
def cumulative_grain (df2):
  """Input a dataframe with column headings in the following order: sample_id, 
      sieve size in mm, sieve size in phi, weight of sediment(g) in each sieve.
        Function will find the total weight and calculate and append the 
        fraction finer and cumulative percent finer to the dataframe """
  weight_sum = df2['weight'].sum() # finds sum of weight column
  for i in range(L): # l = the length of df2
    df2.iloc[i, 4] = (df2.iloc[i,3] / weight_sum) * 100
 
  for i in range(L):
    if i == 0:                     # first row of cumulative finer should always be 100
       df2.iloc[i,5] = 100
    
    else:
      df2.iloc[i,5] = (df2.iloc[i-1,5] - df2.iloc[i,4])

cumulative_grain(df2)
print(df2) # dataframe with fraction finer and cumulative fraction finer added to it

0           sample_id       mm    phi  weight  fraction_finer  \
1   AHA22-CP103GR-BS5  45.3000 -5.501     0.0        0.000000   
2   AHA22-CP103GR-BS5  31.5000 -4.977   298.9        3.140960   
3   AHA22-CP103GR-BS5  22.4000 -4.485     0.0        0.000000   
4   AHA22-CP103GR-BS5  16.0000 -4.000     0.0        0.000000   
5   AHA22-CP103GR-BS5  11.2000 -3.485    11.0        0.115592   
6   AHA22-CP103GR-BS5   8.0000 -3.000    26.2        0.275320   
7   AHA22-CP103GR-BS5   5.6000 -2.485   138.4        1.454362   
8   AHA22-CP103GR-BS5   4.0000 -2.000   431.1        4.530170   
9   AHA22-CP103GR-BS5   2.8000 -1.485   743.1        7.808789   
10  AHA22-CP103GR-BS5   2.0000 -1.000  1036.7       10.894054   
11  AHA22-CP103GR-BS5   1.4000 -0.485  1087.8       11.431033   
12  AHA22-CP103GR-BS5   1.0000  0.000  1241.2       13.043021   
13  AHA22-CP103GR-BS5   0.7100  0.494  1056.4       11.101070   
14  AHA22-CP103GR-BS5   0.5000  1.000  1259.0       13.230071   
15  AHA22-CP103GR-BS5   0

Add fraction finer and cumulative grain size to Google Sheet

In [226]:
# export newly calculate colums to same google sheet
df2_to_sheets = df2[['fraction_finer','cumulative_finer']]
set_with_dataframe(ws,df2_to_sheets,row=1,col=5,include_column_header=True)
#df2

### Cumulative Grain Size Curve

In [227]:
df2["mm"] = pd.to_numeric(df2["mm"])
fig = px.line(df2, x = 'mm', y = 'cumulative_finer', 
               title = df2.iloc[1, 0], labels=dict(cumulative_finer ="Cumulative % Finer", mm =  "Grain Size (mm)"))
fig.update_yaxes(tickvals=[0,10,20,30,40,50,60,70,80,90,100])
#path = '/drive/MyDrive/Classes/Math_120/Midterm_2/'   # have to add path for google colab
#plt.savefig(path + "LGL cumulative grain size curve.png")
fig.show()

### Grain Size Function - Condensed

In [252]:
def grain_size2 (y1):
  """input the desired percent cumalative finer and grain_size 
        will output the corresponding grain size in mm"""
  for i in range (l-1): # has to be one size smaller than the length. 
    if df2.iloc[i+1,5] <= y1 <= df2.iloc[i,5]: 
      m = (df2.iloc[i,5]- df2.iloc[i+1,5]) / (df2.iloc[i,1] - df2.iloc[i+1,1])
      b = df2.iloc[i+1,5] - m * df2.iloc[i+1,1]
      x = (y1-b)/m
      print("D:", y1,'is equal to', x, "mm")

      if y1 < 0:
        raise Exception('y cannot be less than 0')

      if y1 > 100:
        raise Exception('y cannot be greater than 100')         
      
  return y1, x

In [229]:
grain_size2(50)

D: 50 is equal to 1.0825974863035763 mm


(50, 1.0825974863035763)

In [230]:
y_val = [10,16,50,84,90]               # diameter percentiles - will find the corresponding mm

BS_diameter = pd.DataFrame(np.zeros((5,3)))
for i in range(len(y_val)):
  y, x = grain_size2(y_val[i])
  BS_diameter.iloc[i, 0] = df2.iloc[i, 0] # adds the sample name to df
  BS_diameter.iloc[i, 1] = y              # adds the diameter percentile
  BS_diameter.iloc[i, 2] = x              # adds the diameter in mm
BS_diameter

D: 10 is equal to 0.22473848701061067 mm
D: 16 is equal to 0.3520215792169773 mm
D: 50 is equal to 1.0825974863035763 mm
D: 84 is equal to 3.003646346386755 mm
D: 90 is equal to 3.9256842955187703 mm


,0,1,2
0,AHA22-CP103GR-BS5,10.0,0.224738
1,AHA22-CP103GR-BS5,16.0,0.352022
2,AHA22-CP103GR-BS5,50.0,1.082597
3,AHA22-CP103GR-BS5,84.0,3.003646
4,AHA22-CP103GR-BS5,90.0,3.925684


In [231]:
BS_diameter.columns =['sample', 'diameter', 'D_mm']
BS_diameter

,sample,diameter,D_mm
0,AHA22-CP103GR-BS5,10.0,0.224738
1,AHA22-CP103GR-BS5,16.0,0.352022
2,AHA22-CP103GR-BS5,50.0,1.082597
3,AHA22-CP103GR-BS5,84.0,3.003646
4,AHA22-CP103GR-BS5,90.0,3.925684


In [232]:
# Export data to same google sheet
BS_diameter_to_sheets = BS_diameter[['diameter', 'D_mm']]
set_with_dataframe(ws,BS_diameter_to_sheets,row=1,col=7,include_column_header=True)

## Pebble Count

### Dataframe Formatting

To access data, uncomment code for the meadow worksheet you want to access, and then uncomment the code for the specific tab you want to access

In [233]:
# FORMATTING DATAFRAME
# Open Bed Surface Grain Size Sheet
bs0 = gc.open_by_key('1nnCc9azCJu7GB3-T-P6k1NPGTPHQS43JJ9RHv9atFgQ') # Ahart Pebble Count
# bs0 = gc.open_by_key('1MCwy66A2Ez-aNn2nq2CEucFU_d_c8PhHrylfxlegY4U') # Grouse Pebble Count
# bs0 = gc.open_by_key('1OepejnoaO2dM4Z9UTl4Zooy6dyMdv2XxhO8rqBrpJlg') # McCreary Pebble Count

# Open Ahart Tab
ws = bs0.worksheet('AHA22-CP01GR-PC1')

# Open Lower Grouse Tab
# ws = bs0.worksheet('LGR22-CP102R-PC1') # upper gauging station
# ws = bs0.worksheet('LGR22-CPG01R-PC2') # lower gauging station

# Open McCreary Tab
# ws = bs0.worksheet('MCC22-CP50R-PC1')
# ws = bs0.worksheet('MCC22-CPG06R-PC2')
# ws = bs0.worksheet('MCC22-CPG100R-PC3')

# get_all_values gives a list of rows.
rows = ws.get_all_values()
df3 = pd.DataFrame(rows)

# getting rid of column index to so it can be used in Pandas
df3.columns = df3.iloc[0] # update name to new df name
df3 = df3.iloc[1:]  

# only pulling the names of the columns that I want
df3 = df3[["sample_id","mm", 'phi', 'count']]

# drop notes column, uncomment if needed
# df2 = df.drop(columns=['notes'])

# export both dataframes
# path = '/drive/MyDrive/Classes/Math_120/Midterm_2/'
# df.to_csv(path + 'unformatted_dataframe.csv')
# df2.to_csv(path + 'formatted_dataframe.csv')
# df2
# convert columns from strings to numeric
# this code has to be run after the dataframe is formatted 
df3


,sample_id,mm,phi,count
1,AHA22-CP01GR-PC1,362,-8.500,0
2,AHA22-CP01GR-PC1,256,-8.000,25
3,AHA22-CP01GR-PC1,180,-7.492,1
4,AHA22-CP01GR-PC1,128,-7.000,9
5,AHA22-CP01GR-PC1,90,-6.492,10
6,AHA22-CP01GR-PC1,64,-6.000,6
7,AHA22-CP01GR-PC1,45,-5.492,3
8,AHA22-CP01GR-PC1,32,-5.000,1
9,AHA22-CP01GR-PC1,22.6,-4.498,8
10,AHA22-CP01GR-PC1,16,-4.000,9


In [234]:
df3["count"] = pd.to_numeric(df3["count"])
df3["mm"] = pd.to_numeric(df3["mm"])
df3["phi"] = pd.to_numeric(df3["phi"])

### Calculating Cumulative Fraction Finer

In [235]:
# CALCULATE FRACTION FINER AND CUMALATIVE GRAIN SIZE
# find the df length (# of rows) and assign to l, insert into loop
L = len(df3.index)
# Create two empty columns and append to dataframe
df3 ['fraction_finer'] = np.zeros((L,1)) # l = the length of df3
df3 ['cumulative_finer'] = np.zeros((L,1))
def cumulative_grain (df3):
  """Input a dataframe with column headings in the following order: sample_id, 
      sieve size in mm, sieve size in phi, weight of sediment(g) in each sieve.
        Function will find the total weight and calculate and append the 
        fraction finer and cumulative percent finer to the dataframe """
  weight_sum = df3['count'].sum() # finds sum of weight column
  for i in range(l): # l = the length of df3
    df3.iloc[i, 4] = (df3.iloc[i,3] / weight_sum) * 100
 
  for i in range(L):
    if i == 0:                     # first row of cumulative finer should always be 100
       df3.iloc[i,5] = 100
    
    else:
      df3.iloc[i,5] = (df3.iloc[i-1,5] - df3.iloc[i,4])

cumulative_grain(df3)
print(df3)

0          sample_id        mm    phi  count  fraction_finer  cumulative_finer
1   AHA22-CP01GR-PC1  362.0000 -8.500      0             0.0             100.0
2   AHA22-CP01GR-PC1  256.0000 -8.000     25            12.5              87.5
3   AHA22-CP01GR-PC1  180.0000 -7.492      1             0.5              87.0
4   AHA22-CP01GR-PC1  128.0000 -7.000      9             4.5              82.5
5   AHA22-CP01GR-PC1   90.0000 -6.492     10             5.0              77.5
6   AHA22-CP01GR-PC1   64.0000 -6.000      6             3.0              74.5
7   AHA22-CP01GR-PC1   45.0000 -5.492      3             1.5              73.0
8   AHA22-CP01GR-PC1   32.0000 -5.000      1             0.5              72.5
9   AHA22-CP01GR-PC1   22.6000 -4.498      8             4.0              68.5
10  AHA22-CP01GR-PC1   16.0000 -4.000      9             4.5              64.0
11  AHA22-CP01GR-PC1   11.0000 -3.459     12             6.0              58.0
12  AHA22-CP01GR-PC1    8.0000 -3.000      9        

In [236]:
# Adds newly calculated fraction_finer and cumulative_finer columns to same google sheet
df3_to_sheets = df3[['fraction_finer','cumulative_finer']]
set_with_dataframe(ws,df3_to_sheets,row=1,col=5,include_column_header=True)

### Cumulative Grain Size Curve

In [237]:
df3["mm"] = pd.to_numeric(df3["mm"])
fig = px.line(df3, x = 'mm', y = 'cumulative_finer', 
               title = df3.iloc[1, 0], labels=dict(cumulative_finer ="Cumulative % Finer", mm =  "Grain Size (mm)"))
fig.update_yaxes(tickvals=[0,10,20,30,40,50,60,70,80,90,100])
# path = '/drive/MyDrive/Classes/Math_120/Midterm_2/'   # have to add path for google colab
# plt.savefig(path + "LGL cumulative grain size curve.png")
fig.show()

### Grain Size Function

In [250]:
def grain_size3 (y1):
  """input the desired percent cumalative finer and the grain_size function
        will output the corresponding grain size in mm"""
  for i in range (L-1): # has to be one size smaller than the length. 
    if df3.iloc[i+1,5] <= y1 <= df3.iloc[i,5]: 
      m = (df3.iloc[i,5]- df3.iloc[i+1,5]) / (df3.iloc[i,1] - df3.iloc[i+1,1])
      b = df3.iloc[i+1,5] - m * df3.iloc[i+1,1]
      x = (y1-b)/m
      print("D:", y1,'is equal to', x, "mm")  

      if y1 < 0:
        raise Exception('y cannot be less than 0')

      if y1 > 100:
        raise Exception('y cannot be greater than 100')      
      
  return y1, x

In [239]:
grain_size3(50)

D: 50 is equal to 7.066666666666666 mm


(50, 7.066666666666666)

In [240]:
y_val = [10,16,50,84,90]               # diameter percentiles - will find the corresponding mm

PC_diameter = pd.DataFrame(np.zeros((5,3)))
for i in range(len(y_val)):
  y, x = grain_size3(y_val[i])
  PC_diameter.iloc[i, 0] = df3.iloc[i, 0] # adds the sample name to df
  PC_diameter.iloc[i, 1] = y              # adds the diameter percentile
  PC_diameter.iloc[i, 2] = x              # adds the diameter in mm
PC_diameter

D: 10 is equal to 1.0220683544303797 mm
D: 16 is equal to 1.6277493670886074 mm
D: 50 is equal to 7.066666666666666 mm
D: 84 is equal to 145.33333333333337 mm
D: 90 is equal to 277.2 mm


,0,1,2
0,AHA22-CP01GR-PC1,10.0,1.022068
1,AHA22-CP01GR-PC1,16.0,1.627749
2,AHA22-CP01GR-PC1,50.0,7.066667
3,AHA22-CP01GR-PC1,84.0,145.333333
4,AHA22-CP01GR-PC1,90.0,277.200000


In [241]:
PC_diameter.columns =['sample', 'diameter', 'D_mm']
PC_diameter

,sample,diameter,D_mm
0,AHA22-CP01GR-PC1,10.0,1.022068
1,AHA22-CP01GR-PC1,16.0,1.627749
2,AHA22-CP01GR-PC1,50.0,7.066667
3,AHA22-CP01GR-PC1,84.0,145.333333
4,AHA22-CP01GR-PC1,90.0,277.200000


In [242]:
# Outputs data to the same google worksheet
PC_diameter_to_sheets = PC_diameter[['diameter', 'D_mm']]
set_with_dataframe(ws,PC_diameter_to_sheets,row=1,col=7,include_column_header=True)
#df3

# Working Code

Below are code ideas that I don't want to delete, but don't have a use for right now. 

In [243]:
#from google.colab import files
#uploaded = files.upload()

In [244]:
# Original Function
# CUMULATIVE GRAIN SIZE FUNCTION
def grain_size (y):
  """input the desired percent cumalative finer and grain_size 
        will output the corresponding grain size in mm"""
  for i in range (14):
    if df3.iloc[i+1,5] <= y <= df3.iloc[i,5]: 
      m = (df3.iloc[i,5]- df3.iloc[i+1,5]) / (df3.iloc[i,1] - df3.iloc[i+1,1])
      b = df3.iloc[i+1,5] - m * df3.iloc[i+1,1]
      x = (y-b)/m
      return(x)
      
    if y < 0:
      raise Exception('y cannot be less than 0')

    if y > 100:
      raise Exception('y cannot be greater than 100')
     

grain_size(90)

277.2